In [2]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer, TfidfTransformer
from sklearn.datasets import load_files
import pandas as pd
import numpy as np
from function import str_to_list
import re

In [3]:
df = pd.read_csv('csv/new_data.csv')
size = df.shape[0]


Data Preparation

In [4]:
def text_process(text):
    text = re.sub("\[|\]|'|,"," ",text)
    return text
    
df['text_replace'] = df['text'].apply(text_process)
df

,post_type,text,text_replace
0,3,"['หา', 'สัญญา', 'หอ', 'มจพ', 'ย้าย', 'ปลาย', '...",หา สัญญา หอ มจพ ย้าย ปลาย ...
1,1,"['ขาย', 'ตั๋วหนัง', 'black', 'panther', 'โรง',...",ขาย ตั๋วหนัง black panther โรง ...
2,1,"['อนุญาต', 'ขาย', 'ขาย', 'ตู้', 'เย็น', 'Midea...",อนุญาต ขาย ขาย ตู้ เย็น Midea...
3,1,"['XX', 'คน', 'เช่า', 'XX', 'อนุญาต', 'ประกาศ',...",XX คน เช่า XX อนุญาต ประกาศ ...
4,1,"['ขาย', 'สัญญา', 'หอ', 'หมู่บ้าน', 'สิ', 'ริน'...",ขาย สัญญา หอ หมู่บ้าน สิ ริน ...
...,...,...,...
1098,3,"['หา', 'สัญญา', 'หอ', 'S9', 'รีบ', 'เดือน']",หา สัญญา หอ S9 รีบ เดือน
1099,3,"['หนังสือ', 'สอบ', 'ข้อ', 'มจพ', 'แนะนำ', 'แหล...",หนังสือ สอบ ข้อ มจพ แนะนำ แหล...
1100,3,"['ประกาศ', 'ชื่อ', 'น้า']",ประกาศ ชื่อ น้า
1101,3,"['PnetPE', 'เรื่อง', 'เรียน', 'สังคม', 'พี่', ...",PnetPE เรื่อง เรียน สังคม พี่ ...


In [5]:
X = df['text_replace']
y = df["post_type"]

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.25, random_state=0)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=20,decode_error="ignore")
X_train_vectorized = vectorizer.fit_transform(X_train)

Built model

In [15]:
from sklearn.naive_bayes import MultinomialNB
cls = MultinomialNB(alpha=1.0, fit_prior=True)
# transform the list of text to tf-idf before passing it to the model
cls.fit(vectorizer.fit_transform(X_train), y_train)
 
from sklearn.metrics import classification_report, accuracy_score
 
y_pred = cls.predict(vectorizer.fit_transform(X_test))
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.5217391304347826
              precision    recall  f1-score   support

           0       0.50      0.23      0.32        65
           1       0.50      0.61      0.55        93
           3       0.55      0.61      0.58       118

    accuracy                           0.52       276
   macro avg       0.52      0.48      0.48       276
weighted avg       0.52      0.52      0.51       276



In [16]:
import pickle

filename = "./model/check_type.sav"
filenamevec = "./model/vectorizer.sav"

#save model
pickle.dump(cls,open(filename,"wb"))
pickle.dump(vectorizer,open(filenamevec,"wb"))
